# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib as plt
from itertools import combinations
import re

All data and preprocessing results are stored under `.data/`

In [1]:
data_path='data/' 
exp_id='v0'

Create a code mapping dictionary

In [3]:
codes={}

## Drug's target

Load drug's target (from CTDbase)

In [5]:
gene_drug=pd.read_csv(data_path+'CTD/drug-gene-CTD_C000657245_ixns_20200703223915.tsv', sep='\t')

In [6]:
gene_drug.head()

,Chemical Name,Chemical ID,CAS RN,Gene Symbol,Gene ID,Interaction,Interaction Actions,Reference Count,Organism Count
0,Sirolimus,D020123,53123-88-9,A2M,2,Sirolimus results in increased expression of A...,increases^expression,1,1
1,Ivermectin,D007559,70288-86-7,A2ML1.L,100189580,Ivermectin results in increased expression of ...,increases^expression,1,1
2,baricitinib,C000596027,NaN,AAK1,22848,baricitinib results in decreased activity of A...,decreases^activity,1,1
3,Choline,D002794,62-49-7,AARD,441376,[Methionine deficiency co-treated with Choline...,affects^cotreatment|decreases^methylation,1,1
4,Ivermectin,D007559,70288-86-7,AASS.L,444409,Ivermectin results in decreased expression of ...,decreases^expression,1,1


Save the drug name and MeSH ID mapping for later use

In [7]:
codes['drugname2mesh']={row[0].upper():row[1] for idx, row in gene_drug[['Chemical Name','Chemical ID']].drop_duplicates().iterrows()}
codes['mesh2drugname']={row[0].upper():row[1] for idx, row in gene_drug[['Chemical ID','Chemical Name']].drop_duplicates().iterrows()}

In [10]:
#gene_drug=gene_drug.loc[gene_drug['Gene Symbol'].isin(codes['gene_symbol2id'].keys())]

In [11]:
gene_drug=gene_drug[['Gene ID', 'Chemical Name']].drop_duplicates()
gene_drug['Chemical Name']=gene_drug['Chemical Name'].apply(lambda x: x.upper() if type(x)==str else x)

gene_drug['Gene ID']=gene_drug['Gene ID'].apply(lambda x: 'gene_'+str(x))
gene_drug['Chemical Name']=gene_drug['Chemical Name'].apply(lambda x: 'drug_'+x)

gene_drug.drop_duplicates(inplace=True)


## Pathways

Load pathways from CTDbase

In [63]:
pathways=pd.read_csv(data_path+'CTD/pathways-CTD_C000657245_pathways_20200703225429.tsv',sep='\t')
path_sim=pd.concat([pd.DataFrame(list(combinations(pathway,2,)),columns=['gene1','gene2']) for pathway in pathways['Association inferred via'].apply(lambda x: x.split('|') if '|' in x else None).dropna().values]).drop_duplicates()

Load pathways from KEGG. The files were preprocessed as a pairwise gene

In [65]:
path_sim_kegg=pd.read_csv(data_path+'biology-database/KegglinkevaluationPPPN_1', header=None,sep='\t')
path_sim_kegg.columns=['gene1', 'gene2', 'positive']
path_sim_kegg.replace('PP', 1, inplace=True)
path_sim_kegg.replace('PN', 0, inplace=True)

path_sim_kegg=path_sim_kegg.loc[path_sim_kegg['positive']==1, ['gene1','gene2']]

gene_name =pd.read_excel(data_path+'biology-database/All_Human_Protein_Coding_Genes_3_27_2020.xlsx')
gene_dict= {row['Gene Id']:row['Gene Symbol'] for _, row in gene_name[['Gene Id','Gene Symbol']].iterrows()}

In [68]:
path_sim_kegg.dropna()

,gene1,gene2
0,10725,7124
1,10725,1437
2,4772,7124
3,4772,1437
4,4773,7124
...,...,...
29503,8733,84720
29504,8733,80055
29505,23556,55650
29506,2822,284098


In [69]:
path_sim_kegg['gene1']=path_sim_kegg['gene1'].apply(lambda x: gene_dict.get(x))
path_sim_kegg['gene2']=path_sim_kegg['gene2'].apply(lambda x: gene_dict.get(x))

In [70]:
path_sim_kegg.dropna(inplace=True)

Load PHARMAKB [pathways](https://www.pharmgkb.org/page/COVID)

In [71]:
pathway_ace_inhibitor=list(set(['ATP6AP2', 'MAPK1', 'AGTR2', 'ATP6AP2', 'REN', 'MAS1', 'TGFB1', 'MAPK3', 'ATP6AP2', 'MAPK3',
                       'AGTR1','TGFB1', 'MAPK1', 'NOS3', 'BDKRB2', 'BDKRB2', 'BDKRB1', 'NR3C2', 'CYP11B2', 'AGTR1', 'CYP11B2', 'AGTR1', 'AGT', 'KNG1', 'CYP11B2', 'ACE']))
pathway_fluv=['CYP1A2','CYP2C19','CYP3A']
pathway_losartan=list(set(['AGTR1','CYP2C9',"CYP3A4",'CYP2C9',"CYP3A4",'CYP2C9',"CYP3A4", 'CYP2C9',"CYP3A4", 'UGT1A1',"UGT2B7"]))

In [72]:
path_sim=pd.concat([path_sim]+[path_sim_kegg]+[pd.DataFrame(list(combinations(pathway,2,)),columns=['gene1','gene2']) for pathway in [pathway_ace_inhibitor,pathway_fluv,pathway_losartan]])

In [73]:
path_sim['gene1']=path_sim['gene1'].apply(lambda x: codes['gene_symbol2id'].get(x))
path_sim['gene2']=path_sim['gene2'].apply(lambda x: codes['gene_symbol2id'].get(x))
path_sim.dropna(inplace=True)
path_sim['gene1']=path_sim['gene1'].apply(lambda x: 'gene_'+str(int(x)))
path_sim['gene2']=path_sim['gene2'].apply(lambda x: 'gene_'+str(int(x)))
path_sim.drop_duplicates(inplace=True)

In [79]:
len(path_sim)

13423

## Phenotypes

Load phenotypes from CTDbase

In [81]:
phenotypes=pd.read_csv(data_path+'CTD/phenotype-drug-gene-CTD_C000657245_diseases_20200703224649.tsv', sep='\t')

In [82]:
phenotypes.head()

,Phenotype Term Name,Phenotype Term ID,Disease Name,Disease ID,Chemical Inference Network,Gene Inference Network,References
0,cell proliferation,GO:0008283,COVID-19,MESH:C000657245,Azithromycin|betulinic acid|cepharanthine|Chlo...,NaN,162
1,apoptotic process,GO:0006915,COVID-19,MESH:C000657245,Acetazolamide|Azithromycin|betulinic acid|ceph...,BTK|IL1B|IL2RA|TNF,290
2,viral life cycle,GO:0019058,COVID-19,MESH:C000657245,(2-tert-butoxy-1-(2-cyclohexyl-1-(1-formyl-2-(...,NaN,48
3,cell death,GO:0008219,COVID-19,MESH:C000657245,betulinic acid|Chloroquine|Dactinomycin|Iverme...,NaN,101
4,positive regulation of cell death,GO:0010942,COVID-19,MESH:C000657245,Active Hexose Correlated Compound|betulinic ac...,IL1B,40


In [83]:
codes['phenotype_id_to_name']={row[0]:row[1] for idx, row in phenotypes[['Phenotype Term ID','Phenotype Term Name']].drop_duplicates().iterrows()}

Drugs and phenotypes

In [84]:
drug_phenotype=phenotypes['Chemical Inference Network'].dropna().apply(lambda x: x.split('|')).apply(pd.Series).merge(phenotypes['Phenotype Term ID'],left_index=True, right_index=True).melt(id_vars=['Phenotype Term ID'],value_name='drug').drop('variable', axis=1).dropna()
drug_phenotype['drug']=drug_phenotype['drug'].apply(lambda x: x.upper())
drug_phenotype.dropna(inplace=True)

drug_phenotype['Phenotype Term ID']=drug_phenotype['Phenotype Term ID'].apply(lambda x: 'phenotype_'+x)
drug_phenotype['drug']=drug_phenotype['drug'].apply(lambda x: 'drug_'+x)
drug_phenotype=drug_phenotype[['drug', 'Phenotype Term ID']]


Genes and phenotypes

In [86]:
gene_phenotype=phenotypes['Gene Inference Network'].dropna().apply(lambda x: x.split('|')).apply(pd.Series).merge(phenotypes['Phenotype Term ID'],left_index=True, right_index=True).melt(id_vars=['Phenotype Term ID'],value_name='gene').drop('variable', axis=1).dropna()
gene_phenotype['Phenotype Term ID']=gene_phenotype['Phenotype Term ID'].apply(lambda x: 'phenotype_'+x)

gene_phenotype['gene']=gene_phenotype['gene'].apply(lambda x: codes['gene_symbol2id'].get(x))
gene_phenotype.dropna(inplace=True)
gene_phenotype['gene']=gene_phenotype['gene'].apply(lambda x: 'gene_'+str(int(x)))
gene_phenotype=gene_phenotype[['gene', 'Phenotype Term ID']]

In [95]:
len(set(gene_drug['Chemical Name'].values).intersection(set(drug_phenotype['drug'].values)))

31

In [96]:
len(set(drug_phenotype['drug'].values))

36

## Baits and Prey

Load SARS-CoV-2 baits and host gene interaction from [Gorden et al. Nature 2020](https://www.nature.com/articles/s41586-020-2286-9#Sec36)

In [98]:
baits_prey=pd.read_csv(data_path+'biology-database/baits-prey-mist.csv')

In [99]:
baits_prey=baits_prey[['Bait', 'PreyGene']]

In [100]:
baits_prey['Bait'].nunique(), baits_prey['PreyGene'].nunique()

(27, 332)

In [101]:
baits_prey['Bait']=baits_prey['Bait'].apply(lambda x: 'bait_'+x)
baits_prey['PreyGene']=baits_prey['PreyGene'].apply(lambda x: codes['gene_symbol2id'].get(x))
baits_prey.dropna(inplace=True)
baits_prey['PreyGene']=baits_prey['PreyGene'].apply(lambda x: 'gene_'+str(int(x)))

In [108]:
#size of drug target, pathway, host gene, phenotype-related genes
gene_drug['Gene ID'].nunique(),len(set(path_sim[['gene1','gene2']].values.ravel())),len(set(baits_prey['PreyGene'].unique())), gene_phenotype['gene'].nunique()

(4427, 1762, 330, 18)

In [109]:
#number of intersection between host genes and drug target
len(set(baits_prey['PreyGene'].unique()).intersection(gene_drug['Gene ID'].unique()))

94

In [110]:
#intersection between target and pathways
len(set(gene_drug['Gene ID'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())))

733

In [111]:
#intersection between pathways and host gene
len(set(baits_prey['PreyGene'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())))

25

In [112]:
#intersection between pathways, target genes, host gene
len(set(baits_prey['PreyGene'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())).intersection(set(gene_drug['Gene ID'].unique()) ))


10

## Item2idx mapping

Map all the entities (drugs, genes, phenotypes, baits) to ID

In [4]:
from sklearn.preprocessing import LabelEncoder

In [114]:
gene_drug.columns=['node1', 'node2']# gene, drug
path_sim.columns=['node1', 'node2']# gene1, gene2
baits_prey.columns=['node1','node2']#bait, preygene
gene_phenotype.columns=['node1', 'node2'] #gene, phenotype
drug_phenotype.columns=['node1','node2']#drug, phenotye

In [115]:
gene_drug['type']='gene-drug'
path_sim['type']='gene-gene'
baits_prey['type']='bait-gene'
gene_phenotype['type']='gene-phenotype'
drug_phenotype['type']='drug-phenotype'

edge_index=pd.concat([gene_drug, path_sim, baits_prey, gene_phenotype, drug_phenotype])

edge_index['node1']=edge_index['node1'].astype(str)
edge_index['node2']=edge_index['node2'].astype(str)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Label Encoders

In [118]:
le=LabelEncoder()
le.fit(np.concatenate((edge_index['node1'], edge_index['node2'])))

LabelEncoder()

In [119]:
edge_index['node1']=le.transform(edge_index['node1'])
edge_index['node2']=le.transform(edge_index['node2'])

In [120]:
len(le.classes_)

10624

## Import pre-trained embedding

Obtain pre-trained entity embedding from [DRKG](https://github.com/gnn4dr/DRKG)

In [122]:
import csv

In [ ]:
!wget https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz

In [123]:
#Get pretrained embedding
entity_emb=np.load(data_path+'DRKG/embed/DRKG_TransE_l2_entity.npy')
emb_size=entity_emb.shape[1]

In [124]:
entity_idmap_file = data_path+'DRKG/embed/entities.tsv'
relation_idmap_file = data_path+'DRKG/embed/relations.tsv'

In [125]:
baits_drkg=['Disease::'+entity.split('_')[1] for entity  in le.classes_ if entity.split('_')[0]=='bait']
gene_drkg = ['Gene::'+entity.split('_')[1] for entity in le.classes_ if entity.split('_')[0]=='gene']
phenotype_drkg=['Biological Process::'+entity.split('_')[1] for entity in le.classes_ if entity.split('_')[0]=='phenotype']

Map the DRKG's DrugBank ID to MeSH ID

In [126]:
drugname2external=pd.concat([pd.read_csv(data_path+'CTD/alldrugbank.csv', index_col=0).rename(columns={'drugbank_id':'id'}),
           pd.read_csv(data_path+'CTD/nondrugbank.csv', index_col=0).rename(columns={'chembl':'id'})]).groupby('drugname', as_index=False).first()
drugname2id={row[0].upper():row[1] for _, row in drugname2external[['drugname', 'id']].iterrows()}
drug_drkg = ['Compound::'+drugname2id.get(entity.split('_')[1],'') for entity in le.classes_ if entity.split('_')[0]=='drug']

Get drugname/disease name to entity ID mappings

In [127]:

entity_map = {}
entity_id_map = {}
relation_map = {}
with open(entity_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        entity_map[row_val['name']] = int(row_val['id'])
        entity_id_map[int(row_val['id'])] = row_val['name']
        
with open(relation_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        relation_map[row_val['name']] = int(row_val['id'])
        
# handle the ID mapping
bait_ids = []
gene_ids = []
drug_ids = []
phenotype_ids = []

    
for bait in baits_drkg:
    bait_ids.append(entity_map.get(bait))

for gene in gene_drkg:
    gene_ids.append(entity_map.get(gene))
    
for drug in drug_drkg:
    drug_ids.append(entity_map.get(drug))
    
for phenotype in phenotype_drkg:
    phenotype_ids.append(entity_map.get(phenotype))


In [128]:
bait_emb=np.array([entity_emb[bait_id] if bait_id is not None else np.zeros(emb_size) for bait_id in bait_ids ])
drug_emb=np.array([entity_emb[drug_id] if drug_id is not None else np.zeros(emb_size) for drug_id in drug_ids ])
gene_emb=np.array([entity_emb[gene_id] if gene_id is not None else np.zeros(emb_size) for gene_id in gene_ids ])
phenotype_emb=np.array([entity_emb[phenotype_id] if phenotype_id is not None else np.zeros(emb_size) for phenotype_id in phenotype_ids ])


In [129]:
#How many missing in drugs?
len(drug_ids),len([gene_id for gene_id in drug_ids if gene_id is not None])

(3635, 1138)

In [130]:
#How many missing in genes?
len(gene_ids),len([gene_id for gene_id in gene_ids if gene_id is not None])

(5677, 5666)

In [131]:
#How many missing in phenotypes?
len(phenotype_ids),len([gene_id for gene_id in phenotype_ids if gene_id is not None])

(1285, 970)

The pre-trained embedding is now serived as a node feature

In [132]:
node_features=np.concatenate((bait_emb, drug_emb, gene_emb, phenotype_emb))

## Save as pickle

In [133]:
edge_index.to_pickle(data_path+'edge_index_'+exp_id+'.pkl')
pickle.dump(le, open(data_path+'LabelEncoder_'+exp_id+'.pkl','wb'))
pickle.dump(node_features, open(data_path+'node_feature_'+exp_id+'.pkl', 'wb'))
pickle.dump(codes, open(data_path+'codes_'+exp_id+'.pkl','wb'))